In [22]:
import pandas as pd
base = pd.read_csv('credit_data.csv')
base = base.drop('i#clientid', axis = 1)
base = base.dropna()
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
X = base.drop('c#default', axis = 1)
y = base['c#default']
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [23]:
neuronios_entrada = 3
neuronios_oculta = 2
neuronios_saida = neuronios_entrada
# 3 -> 2 -> 3

In [24]:
import tensorflow as tf

In [25]:
xph = tf.placeholder(tf.float32, shape = [None, neuronios_entrada])

In [26]:
# fully_connected é uma camada densa (todos neuronios de uma camada se conectam aos neuronios da outra camada)

from tensorflow.contrib.layers import fully_connected
camada_oculta = fully_connected(inputs = xph, num_outputs = neuronios_oculta, activation_fn = None)
camada_saida = fully_connected(inputs = camada_oculta, num_outputs = neuronios_saida)

In [27]:
erro = tf.losses.mean_squared_error(labels = xph, predictions = camada_saida)
otimizador = tf.train.AdamOptimizer(0.01)
treinamento = otimizador.minimize(erro)

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(1000):
        custo, _ = sess.run([erro, treinamento], feed_dict = {xph: X})
        if epoca % 100 == 0:
            print('erro: ' + str(custo))
    X2d_encode = sess.run(camada_oculta, feed_dict = {xph: X})
    X3d_decode = sess.run(camada_saida, feed_dict = {xph: X})

erro: 1.3667204
erro: 0.67969686
erro: 0.64485055
erro: 0.5651173
erro: 0.5617514
erro: 0.5605436
erro: 0.5602868
erro: 0.5602472
erro: 0.56023926
erro: 0.56023836


In [29]:
X2d_encode.shape

(1997, 2)

In [30]:
X3d_decode.shape

(1997, 3)

In [31]:
X2 = scaler_x.inverse_transform(X)
X2

array([[6.61559251e+04, 5.90170151e+01, 8.10653213e+03],
       [3.44151540e+04, 4.81171531e+01, 6.56474502e+03],
       [5.73171701e+04, 6.31080495e+01, 8.02095330e+03],
       ...,
       [4.43114493e+04, 2.80171669e+01, 5.52278669e+03],
       [4.37560566e+04, 6.39717958e+01, 1.62272260e+03],
       [6.94365796e+04, 5.61526170e+01, 7.37883360e+03]])

In [32]:
X3d_decode2 = scaler_x.inverse_transform(X3d_decode)
X3d_decode2

array([[6.2039652e+04, 5.8925270e+01, 9.2395098e+03],
       [4.7624715e+04, 4.8549717e+01, 4.4454878e+03],
       [5.8413625e+04, 6.3188019e+01, 7.7037715e+03],
       ...,
       [5.0214754e+04, 4.0807560e+01, 4.4454878e+03],
       [4.5333863e+04, 6.3689056e+01, 4.4454878e+03],
       [6.2197207e+04, 5.5922123e+01, 9.2613369e+03]], dtype=float32)

In [33]:
from sklearn.metrics import mean_absolute_error
mae_income = mean_absolute_error(X2[:,0], X3d_decode2[:,0])
mae_income

9393.910851676059

In [34]:
mae_age = mean_absolute_error(X2[:,1], X3d_decode2[:,1])
mae_age

5.96122534935627

In [35]:
mae_loan = mean_absolute_error(X2[:,2], X3d_decode2[:,2])
mae_loan

1803.728516664474

In [36]:
X_encode = pd.DataFrame({'atributo1': X2d_encode[:,0], 'atributo2': X2d_encode[:,1], 'classe': y})

In [37]:
X_encode.head()

,atributo1,atributo2,classe
0,-0.012378,-2.174335,0
1,-0.929145,-0.559670,0
2,-0.861266,-2.405181,0
3,-0.375955,-0.433502,0
4,3.911193,1.459635,1


In [38]:
import tensorflow as tf
colunas = [tf.feature_column.numeric_column(key = column) for column in X_encode.columns]
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X_encode, y, test_size = 0.3, random_state = 0)
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento,
                                                        y = y_treinamento,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = y_teste,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Douglas\\AppData\\Local\\Temp\\tmpwxlmwl8u', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026630B7F848>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Var

C:\Users\Douglas\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Douglas\AppData\Local\Temp\tmpwxlmwl8u\model.ckpt.
INFO:tensorflow:loss = 6.8066635, step = 1
INFO:tensorflow:global_step/sec: 432.868
INFO:tensorflow:loss = 0.56572443, step = 101 (0.233 sec)
INFO:tensorflow:global_step/sec: 671.058
INFO:tensorflow:loss = 0.1340839, step = 201 (0.148 sec)
INFO:tensorflow:global_step/sec: 680.259
INFO:tensorflow:loss = 0.2630666, step = 301 (0.148 sec)
INFO:tensorflow:global_step/sec: 680.224
INFO:tensorflow:loss = 0.13954291, step = 401 (0.147 sec)
INF

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-17T16:31:28Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Douglas\AppData\Local\Temp\tmpwxlmwl8u\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2020-08-17-16:31:30
INFO:tensorflow:Saving dict for global step 1000: accuracy = 1.0, accu

In [39]:
metricas_teste

{'accuracy': 1.0,
 'accuracy_baseline': 0.8635,
 'auc': 1.0,
 'auc_precision_recall': 1.0,
 'average_loss': 0.0020813064,
 'label/mean': 0.1365,
 'loss': 0.016650451,
 'precision': 1.0,
 'prediction/mean': 0.13769643,
 'recall': 1.0,
 'global_step': 1000}